In [1]:
# This notebook tests the new function calling feature of GPT-3.5-turbo.

# Set up chatgpt api
import openai
import json
import os
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
GPT_KEY = os.environ['OPENAI_API_KEY']
GPT_MODEL = "gpt-3.5-turbo-0613"


## 1. Parse information

In [4]:
completion = openai.ChatCompletion.create(
    model=GPT_MODEL,
    messages=[{"role": "user", "content": "What's the weather like in Boston?"}],
    functions=[
    {
        "name": "get_current_weather",
        "description": "Get the current weather in a given location",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city and state, e.g. San Francisco, CA",
                },
                "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
            },
            "required": ["location"],
        },
    }
],
function_call="auto",
)

In [5]:
reply_content = completion.choices[0]
reply_content


<OpenAIObject at 0x1496ddea0> JSON: {
  "finish_reason": "function_call",
  "index": 0,
  "message": {
    "content": null,
    "function_call": {
      "arguments": "{\n\"location\": \"Boston\"\n}",
      "name": "get_current_weather"
    },
    "role": "assistant"
  }
}

In [6]:
reply_content = completion.choices[0].message

funcs = reply_content.to_dict()['function_call']['arguments']
funcs = json.loads(funcs)
print(funcs)
print(funcs['location'])


{'location': 'Boston'}
Boston


## 2. Get structured answers from ChatGPT

In [31]:
data = []
with open("data/MultiRC/val.jsonl", "r") as f:
    data_in = f.readlines()
for line in data_in:
    data.append(json.loads(line))

In [103]:
content_good = """Text:
[beginning of text]
Einstein and Maric married in January 1903. In May 1904, the couple\'s first son, Hans Albert Einstein, was born in Bern, Switzerland. Their second son, Eduard, was born in Zurich in July 1910. In 1914, the couple separated; Einstein moved to Berlin and his wife remained in Zurich with their sons. They divorced on 14 February 1919, having lived apart for five years. Eduard, whom his father called "Tete" (for petit), had a breakdown at about age 20 and was diagnosed with schizophrenia. His mother cared for him and he was also committed to asylums for several periods, including full-time after her death. The marriage with Maric does not seem to have been very happy. In letters revealed in 2015, Einstein wrote to his early love, Marie Winteler, about his marriage and his still strong feelings for Marie. In 1910 he wrote to her that "I think of you in heartfelt love every spare minute and am so unhappy as only a man can be" while his wife was pregnant with their second child. Einstein spoke about a "misguided love" and a "missed life" regarding his love for Marie. Einstein married Elsa Lowenthal on 2 June 1919, after having had a relationship with her since 1912. She was a first cousin maternally and a second cousin paternally. In 1933, they emigrated to the United States. In 1935, Elsa Einstein was diagnosed with heart and kidney problems; she died in December 1936. 
[end of text]
Summary:
[beginning of summary]
Questions: Where was Elsa Einstein most likely living when she was diagnosed with heart and kidney problems?
Answer: United States
[end of summary]
"""
content_bad = """Text:
[beginning of text]
Einstein and Maric married in January 1903. In May 1904, the couple\'s first son, Hans Albert Einstein, was born in Bern, Switzerland. Their second son, Eduard, was born in Zurich in July 1910. In 1914, the couple separated; Einstein moved to Berlin and his wife remained in Zurich with their sons. They divorced on 14 February 1919, having lived apart for five years. Eduard, whom his father called "Tete" (for petit), had a breakdown at about age 20 and was diagnosed with schizophrenia. His mother cared for him and he was also committed to asylums for several periods, including full-time after her death. The marriage with Maric does not seem to have been very happy. In letters revealed in 2015, Einstein wrote to his early love, Marie Winteler, about his marriage and his still strong feelings for Marie. In 1910 he wrote to her that "I think of you in heartfelt love every spare minute and am so unhappy as only a man can be" while his wife was pregnant with their second child. Einstein spoke about a "misguided love" and a "missed life" regarding his love for Marie. Einstein married Elsa Lowenthal on 2 June 1919, after having had a relationship with her since 1912. She was a first cousin maternally and a second cousin paternally. In 1933, they emigrated to the United States. In 1935, Elsa Einstein was diagnosed with heart and kidney problems; she died in December 1936. 
[end of text]
Summary:
[beginning of summary]
Questions: Where was Elsa Einstein most likely living when she was diagnosed with heart and kidney problems?
Answer: In Zurich
[end of summary]
"""
# In Einstein's heart
# ASSESSMENT_CHOICES = ["contradiction", "entailment", "neutral"]
ASSESSMENT_CHOICES = ["contradiction", "entailment"]

In [107]:
completion = openai.ChatCompletion.create(
    model=GPT_MODEL,
    messages=[{"role": "user", "content": content_good}],
    functions=[
    {
        "name": "check_contradiction",
        "description": "Check the contradiction between the text and the summary",
        "parameters": {
            "type": "object",
            "properties": {
                # "analysis": {
                #     "type": "string",
                #     "description": "A logical analysis comparing the text and the summary",
                # },
                "assessment": {
                    "type": "string", 
                    "description": "Final assessment of the contradiction between the text and the summary",
                    "enum": ASSESSMENT_CHOICES
                },
            },
            # "required": ["analysis", "assessment"],
            "required": ["assessment"],
        },
    }
],
function_call="auto",
)
args = json.loads(completion["choices"][0]["message"].to_dict()['function_call']["arguments"])
args

{'assessment': 'entailment'}

In [106]:
completion = openai.ChatCompletion.create(
    model=GPT_MODEL,
    messages=[{"role": "user", "content": content_bad}],
    functions=[
    {
        "name": "check_contradiction",
        "description": "Check the contradiction between the text and the summary",
        "parameters": {
            "type": "object",
            "properties": {
                # "analysis": {
                #     "type": "string",
                #     "description": "A short analysis of the text and the summary",
                # },
                "assessment": {"type": "string", "enum": ASSESSMENT_CHOICES},
            },
            "required": ["analysis", "assessment"],
        },
    }
],
function_call="auto",
)
args = json.loads(completion["choices"][0]["message"].to_dict()['function_call']["arguments"])
args

{'assessment': 'contradiction'}

# Use external tools

ChatGPT is powerful, however, it is still limited in many ways like not having access to the internet, limited ability to do math. Using functions, we can enable ChatGPT to use external tools to supplement its capabilities. In fact, this is probably the main use of functions that OpenAI's developer had in mind.

In [108]:
def c_to_f(c):
    return c * 9 / 5 + 32

def f_to_c(f):
    return (f - 32) * 5 / 9